In [0]:
# import lib
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType
from pyspark.sql.types import StringType
import pyspark.sql.functions as f

In [0]:
payments = spark.table("default.staging_payments")
display(payments.limit(5))

payment_id,date,amount,rider_id
2,2019-06-01,9.0,1000
3,2019-07-01,9.0,1000
4,2019-08-01,9.0,1000
5,2019-09-01,9.0,1000
6,2019-10-01,9.0,1000


In [0]:
spark.sql("drop table if exists default.fact_payments")
payments.dropDuplicates(["payment_id"]).write.format("delta").mode("overwrite").saveAsTable("default.fact_payments")

In [0]:
riders = spark.table("default.staging_riders")
display(riders.limit(5))

rider_id,first_name,last_name,address,birthday,account_start_date,account_end_date,is_member
1001,Jennifer,Smith,397 Diana Ferry,1976-08-10,2019-11-01,2020-09-01,True
1002,Karen,Smith,644 Brittany Row Apt. 097,1998-08-10,2022-02-04,null,True
1003,Bryan,Roberts,996 Dickerson Turnpike,1999-03-29,2019-08-26,null,False
1004,Jesse,Middleton,7009 Nathan Expressway,1969-04-11,2019-09-14,null,True
1005,Christine,Rodriguez,224 Washington Mills Apt. 467,1974-08-27,2020-03-24,null,False


In [0]:
spark.sql("drop table if exists default.dim_rider")
riders.dropDuplicates(["rider_id"]).write.format("delta").mode("overwrite").saveAsTable("default.dim_rider")

In [0]:
stations = spark.table("default.staging_stations")
display(stations.limit(5))

station_id,name,latitude,longitude
KA1503000012,Clark St & Lake St,41.88579466666667,-87.63110066666668
637,Wood St & Chicago Ave,41.895634,-87.672069
13216,State St & 33rd St,41.8347335,-87.6258275
18003,Fairbanks St & Superior St,41.89580766666667,-87.62025316666669
KP1705001026,LaSalle Dr & Huron St,41.894877,-87.632326


In [0]:
spark.sql("drop table if exists default.dim_station")
stations.dropDuplicates(["station_id"]).write.format("delta").mode("overwrite").saveAsTable("default.dim_station")

In [0]:
trips = spark.table("default.staging_trips")
display(trips.limit(5))

trip_id,rideable_type,start_at,ended_at,start_station_id,end_station_id,rider_id
0FEFDE2603568365,classic_bike,2021-02-14 17:52:38,2021-02-14 18:12:09,525,16806,47854
E6159D746B2DBB91,electric_bike,2021-02-09 19:10:18,2021-02-09 19:19:10,KA1503000012,TA1305000029,70870
B32D3199F1C2E75B,classic_bike,2021-02-02 17:49:41,2021-02-02 17:54:06,637,TA1305000034,58974
83E463F23575F4BF,electric_bike,2021-02-23 15:07:23,2021-02-23 15:22:37,13216,TA1309000055,39608
BDAA7E3494E8D545,electric_bike,2021-02-24 15:43:33,2021-02-24 15:49:05,18003,KP1705001026,36267


In [0]:
fact_trips = trips.join(riders, trips.rider_id == riders.rider_id,"inner") \
            .withColumn('duration', round((unix_timestamp("ended_at") - unix_timestamp('start_at'))/60)) \
            .withColumn('rider_age', round(datediff( to_date("start_at"), to_date("birthday"))/365)) \
            .select("trip_id", riders.rider_id, "rideable_type", "start_station_id", "end_station_id", "start_at", "ended_at", "duration", "rider_age")
            

In [0]:
display(fact_trips.limit(5))

trip_id,rider_id,rideable_type,start_station_id,end_station_id,start_at,ended_at,duration,rider_age
222BB8E5059252D7,34062,classic_bike,KA1503000064,13021,2021-06-13 09:48:47,2021-06-13 10:07:23,19.0,30.0
1826E16CB5486018,5342,classic_bike,TA1306000010,13021,2021-06-21 22:59:13,2021-06-21 23:04:29,5.0,26.0
3D9B6A0A5330B04D,3714,classic_bike,TA1305000030,13021,2021-06-18 16:06:42,2021-06-18 16:12:02,5.0,26.0
07E82F5E9C9E490F,18793,classic_bike,TA1305000034,13021,2021-06-17 16:46:23,2021-06-17 17:02:45,16.0,19.0
A8E94BAECBF0C2DD,43342,docked_bike,TA1308000009,TA1308000009,2021-06-13 17:36:29,2021-06-13 18:30:39,54.0,28.0


In [0]:
spark.sql("drop table if exists default.fact_trip")
fact_trips.write.format("delta").mode("overwrite").saveAsTable("default.fact_trip")

In [0]:
# Get min and max dates
min_max_dates = trips.selectExpr("MIN(start_at) as min_date", "MAX(ended_at) as max_date").first()

# Extract min and max dates
min_date = min_max_dates["min_date"]
max_date = min_max_dates["max_date"]

# Generate sequence expression
expression = f"sequence(to_date('{min_date}'), to_date('{max_date}'), interval 1 day)"

# Create DataFrame with single row
dim_date = spark.createDataFrame([(1,)], ["date_id"])

# Explode the sequence expression to generate dates
dim_date = dim_date.withColumn("dateinit", explode(expr(expression)))

# Convert 'dateinit' to timestamp
dim_date = dim_date.withColumn("date", to_timestamp(dim_date.dateinit, "yyyy-MM-dd"))

# Extract date components
dim_date = dim_date.withColumn("day", dayofmonth(dim_date.date)) \
                   .withColumn("month", month(dim_date.date)) \
                   .withColumn("quarter", quarter(dim_date.date)) \
                   .withColumn("year", year(dim_date.date)) \
                   .withColumn("day_of_year", dayofyear(dim_date.date)) \
                   .withColumn("day_of_week", dayofweek(dim_date.date)) \
                   .withColumn("date_id", col("date").cast(StringType())) \
                   .drop("dateinit")

# Display the result
display(dim_date.limit(5))

date_id,date,day,month,quarter,year,day_of_year,day_of_week
2021-02-01 00:00:00,2021-02-01T00:00:00.000+0000,1,2,1,2021,32,2
2021-02-02 00:00:00,2021-02-02T00:00:00.000+0000,2,2,1,2021,33,3
2021-02-03 00:00:00,2021-02-03T00:00:00.000+0000,3,2,1,2021,34,4
2021-02-04 00:00:00,2021-02-04T00:00:00.000+0000,4,2,1,2021,35,5
2021-02-05 00:00:00,2021-02-05T00:00:00.000+0000,5,2,1,2021,36,6


In [0]:
spark.sql("drop table if exists default.dim_time")
dim_date.write.format("delta").mode("overwrite").saveAsTable("default.dim_date")